In [ ]:
# @title 🚀 INICIAR GERADOR DEFINITIVO (VERSÃO ESTÁVEL 0.29)
import os, subprocess, time, re, socket, threading, hashlib, base64

# --- CONFIGURAÇÃO DO AMBIENTE ---
print("🧹 1. Limpando processos...")
!pkill -9 cloudflared
!rm -f o.png

print("📦 2. Instalando dependências compatíveis (Aguarde)...")
# Versão 0.29.0 resolve o erro de 'guiders' e mantém compatibilidade
!pip install -q "diffusers==0.29.0" "transformers==4.40.0" "accelerate==0.30.0" flask flask-cors

import torch
from diffusers import StableDiffusionXLPipeline
from flask import Flask, request, jsonify
from flask_cors import CORS

# --- CONFIGURAÇÃO DA IA ---
print("🚀 3. Carregando IA na GPU (SDXL Turbo)...")
try:
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/sdxl-turbo", 
        torch_dtype=torch.float16, 
        variant="fp16",
        use_safetensors=True
    ).to("cuda")
    print("✅ IA Carregada com sucesso!")
except Exception as e:
    print(f"❌ ERRO CRÍTICO: {e}")
    raise

app = Flask(__name__)
CORS(app)
memory = {}

@app.route('/')
def health(): return "✅ Servidor Online"

@app.route('/generate', methods=['POST'])
def generate():
    try:
        data = request.json
        prompt = data.get("prompt", "").lower()
        words = prompt.split()
        entity = words[0] if words else "default"
        
        if entity not in memory:
            memory[entity] = int(hashlib.md5(entity.encode()).hexdigest(), 16) % 10**8
        
        image = pipe(
            prompt=prompt, 
            num_inference_steps=1, 
            guidance_scale=0.0, 
            generator=torch.Generator("cuda").manual_seed(memory[entity])
        ).images[0]
        
        image.save("o.png")
        with open("o.png", "rb") as f:
            return jsonify({"image": base64.b64encode(f.read()).decode('utf-8')})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# --- INICIAR SERVIDOR ---
def run_flask():
    app.run(host='0.0.0.0', port=5000, use_reloader=False)

threading.Thread(target=run_flask).start()
time.sleep(5)

# --- CONFIGURAR CLOUDFLARE ---
print("🌐 4. Ativando Túnel Cloudflare...")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb > /dev/null 2>&1

p = subprocess.Popen(['cloudflared', 'tunnel', '--url', 'http://127.0.0.1:5000'], 
                     stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in p.stdout:
    if ".trycloudflare.com" in line:
        url = re.search(r"https://[a-zA-Z0-9-]+\.trycloudflare\.com", line)
        if url:
            print("\n" + "="*60)
            print(f"🚀 SISTEMA ONLINE: {url.group(0)}")
            print("="*60)
            break